# Skin lesion classification of dermoscopic images using machine learning and convolutional neural network

19 December 2022

https://www.nature.com/articles/s41598-022-22644-9#Tab7

# 전처리

In [4]:
# 표준 라이브러리
import os
import json

# 써드 파티 라이브러리
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# TensorFlow 관련 라이브러리
from tensorflow.keras import models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [5]:
# 데이터 디렉토리 경로
data_dir = 'New_Sample/라벨링데이터/TL01/반려묘/피부/일반카메라/유증상/A2_비듬_각질_상피성잔고리/'

# json 파일 리스트 가져오기
json_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.json')]
json_files.sort()

# 이미지 파일 리스트 가져오기
image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]
image_files.sort()

In [6]:
for json_path, image_path in zip(json_files, image_files):
    # JSON 파일 읽기
    with open(json_path, 'r') as f:
        json_data = json.load(f)

    # metaData 정보 가져오기
    meta_data = json_data['metaData']
    breed = meta_data['breed']
    age = meta_data['age']
    gender = meta_data['gender']
    region = meta_data['region']
    camera_type = meta_data['camera type']
    species = meta_data['species']
    lesions = meta_data['lesions']

    # labelingInfo 정보 가져오기
    labeling_info = json_data['labelingInfo']

    # polygon 정보
    polygon_coordinates = []
    
    # box 정보
    box_location = []
    
    for item in labeling_info:
        if 'polygon' in item:
            polygon = item['polygon']
            location = polygon['location'][0]
            num_coordinates = len(location) // 2
            coordinates = [(location[f'x{i+1}'], location[f'y{i+1}']) for i in range(num_coordinates)]
            polygon_coordinates.append(coordinates)
            
        if 'box' in item:
            box = item['box']
            location = box['location'][0]
            x = location['x']
            y = location['y']
            width = location['width']
            height = location['height']
            box_location.append((x, y, width, height))

    # 결과 출력
    print(f'견종: {breed}')
    print(f'나이: {age}')
    print(f'성별: {gender}')
    print(f'촬영위치: {region}')
    print(f'촬영장비: {camera_type}')
    print(f'반려종: {species}')
    print(f'증상: {lesions}')
    print('폴리곤 좌표:')
    for coords in polygon_coordinates:
        print(f'- {coords}')
    print('박스 위치:')
    for location in box_location:
        print(f'- x: {location[0]}, y: {location[1]}, width: {location[2]}, height: {location[3]}')
    print('=' * 50)
    
    # 결과 이미지 저장 경로 생성
    os.makedirs('cropped_image_polygon', exist_ok=True)
    os.makedirs('cropped_image_box', exist_ok=True)
    
    # 이미지 파일 이름
    image_filename = os.path.basename(image_path)

    # 이미지 불러오기
    image = Image.open(image_path)
    image_np = np.array(image)
    image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    # 폴리곤 좌표로 마스크 생성
    mask = np.zeros(image_bgr.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [np.array(coords)], 255)

    # 이미지 크롭
    cropped_image = cv2.bitwise_and(image_bgr, image_bgr, mask=mask)
    
    # 이미지 리사이징
    cropped_resized_img = cv2.resize(cropped_image, (96, 96))

    # 결과 이미지 저장
    cv2.imwrite(os.path.join('cropped_image_polygon', image_filename), cropped_image)

    # 박스 좌표로 이미지 크롭
    cropped_image = image_bgr[y:y + height, x:x + width]
    
    # 이미지 리사이징
    cropped_resized_img = cv2.resize(cropped_image, (96, 96))

    # 결과 이미지 저장
    cv2.imwrite(os.path.join('cropped_image_box', image_filename), cropped_image)

견종: 페르시안
나이: 10
성별: F
촬영위치: L
촬영장비: IMG
반려종: C
증상: A2
폴리곤 좌표:
- [(619, 730), (587, 745), (473, 749), (455, 638), (477, 529), (577, 467), (645, 495), (619, 730)]
박스 위치:
- x: 455, y: 467, width: 190, height: 282
견종: 페르시안
나이: 3
성별: M
촬영위치: L
촬영장비: IMG
반려종: C
증상: A2
폴리곤 좌표:
- [(1085, 631), (1001, 631), (956, 599), (942, 544), (985, 503), (1031, 499), (1070, 529), (1085, 631)]
박스 위치:
- x: 942, y: 499, width: 143, height: 132
견종: 페르시안
나이: 7
성별: F
촬영위치: H
촬영장비: IMG
반려종: C
증상: A2
폴리곤 좌표:
- [(1384, 867), (1415, 837), (1452, 796), (1445, 744), (1384, 708), (1321, 704), (1272, 744), (1246, 781), (1236, 800), (1237, 802), (1240, 805), (1206, 822), (1158, 863), (1113, 894), (1061, 917), (1034, 956), (1049, 976), (1082, 958), (1154, 915), (1194, 890), (1251, 864), (1288, 856), (1354, 894), (1384, 867)]
박스 위치:
- x: 1034, y: 704, width: 418, height: 272
견종: 노르웨이숲
나이: 10
성별: F
촬영위치: B
촬영장비: IMG
반려종: C
증상: A2
폴리곤 좌표:
- [(576, 985), (644, 975), (674, 956), (689, 928), (678, 884), (665, 853), (624, 833), 

In [ ]:
def load_resized_images(image_path):
    # 저장한 리사이징한 이미지를 불러옵니다.
    resized_images = np.load(image_path)
    return resized_images

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

In [ ]:
# 분류 모델과 하이퍼파라미터를 설정합니다
model_LR = LogisticRegression(random_state=9)
model_LDA = LinearDiscriminantAnalysis(solver='svd')
model_KNN = KNeighborsClassifier(n_neighbors=5)
model_DT = DecisionTreeClassifier(n_estimators=100)
model_RF = RandomForestClassifier(n_estimators=200, random_state=0)
model_GaussianNB = GaussianNB(var_smoothing=1e-09)
model_SVM = SVC(kernel='linear', C=1, random_state=0)

# 분류 모델들을 리스트에 담습니다
models = [model_LR, model_LDA, model_KNN, model_DT, model_RF, model_GaussianNB, model_SVM]

# 각 분류 모델을 학습시키고 예측 결과를 출력합니다
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(f"{model.__class__.__name__}: {score}")

In [ ]:
# Sequential 모델 생성
model = models.Sequential()

# 첫번째 Conv2D 레이어
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(96,96,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))

# 첫번째 Dropout 레이어
model.add(Dropout(0.25))

# 두번째 Conv2D 레이어
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# 두번째 Dropout 레이어
model.add(Dropout(0.25))

# 세번째 Conv2D 레이어
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# 세번째 Dropout 레이어
model.add(Dropout(0.25))

# Flatten 레이어
model.add(Flatten())

# 첫번째 Dense 레이어
model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())

# 두번째 Dropout 레이어
model.add(Dropout(0.5))

# 두번째 Dense 레이어: 최종 출력 레이어
model.add(Dense(units=7, activation='softmax'))

# 모델 컴파일
opt = Adam(lr=0.001, decay=0.00001)
model.compile(loss='categorical_crossentropy', 
              optimizer=opt, 
              metrics=['accuracy'])

# 모델 구조 요약
model.summary()

# 모델 학습
epochs = 150
batch_size = 32

history = model.fit(train_data, epochs=epochs, batch_size=batch_size, validation_data=val_data)

# 모델 평가
test_loss, test_acc = model.evaluate(test_data)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

In [ ]:
# TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 변환된 모델을 파일로 저장
with open('your_model.tflite', 'wb') as f:
    f.write(tflite_model)